In [3]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(r'C:\Users\adnane\Desktop\ORANGECHAT2\darija-data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:\Users\adnane\Desktop\ORANGECHAT2\darija-data\(in)definite.csv
C:\Users\adnane\Desktop\ORANGECHAT2\darija-data\adjectives.csv
C:\Users\adnane\Desktop\ORANGECHAT2\darija-data\adverbs.csv
C:\Users\adnane\Desktop\ORANGECHAT2\darija-data\animals.csv
C:\Users\adnane\Desktop\ORANGECHAT2\darija-data\art.csv
C:\Users\adnane\Desktop\ORANGECHAT2\darija-data\clothes.csv
C:\Users\adnane\Desktop\ORANGECHAT2\darija-data\colors.csv
C:\Users\adnane\Desktop\ORANGECHAT2\darija-data\conjug_past.csv
C:\Users\adnane\Desktop\ORANGECHAT2\darija-data\conjug_present.csv
C:\Users\adnane\Desktop\ORANGECHAT2\darija-data\determiners.csv
C:\Users\adnane\Desktop\ORANGECHAT2\darija-data\economy.csv
C:\Users\adnane\Desktop\ORANGECHAT2\darija-data\education.csv
C:\Users\adnane\Desktop\ORANGECHAT2\darija-data\emotions.csv
C:\Users\adnane\Desktop\ORANGECHAT2\darija-data\environment.csv
C:\Users\adnane\Desktop\ORANGECHAT2\darija-data\family.csv
C:\Users\adnane\Desktop\ORANGECHAT2\darija-data\femalenames.csv
C:\Users\adn

In [5]:
data = {'english':[],
       'darija':[]}
filename = r"C:\Users\adnane\Desktop\ORANGECHAT2\darija-data\sentences.csv"
with open(filename) as file:
    for line in file:
        if '"' in line:
            line1 = line.split('"')
            if len(line1)>=2:
                data['english'].append(line1[1])
                data['darija'].append(line1[len(line1)-2])
        else:
            line2 = line.split(',')
            if len(line2)>=2:
                data['english'].append(line2[0])
                data['darija'].append(line2[1])
            

In [6]:
print(len(data['english']))
print(len(data['darija']))

1037
1037


In [7]:
df = pd.DataFrame.from_dict(data)

In [8]:
df.head()

,english,darija
0,english,darija
1,"They're hiding something, I'm sure!","homa mkhbbyin chi haja, ana mti99en!"
2,It's obvious they're trying to keep their cool.,bayna homa tay7awlo ib9aw mbrrdin.
3,the hotels don't seem very comfortable.,loTilat mabaynach fihom mori7in bzzaf.
4,he is probably about to be laid off by head of...,ghaliban ghayjrriw 3lih mn lkhdma!


In [9]:
english = df['english']
darija = df['darija']
tf.convert_to_tensor(english)
tf.convert_to_tensor(darija)

<tf.Tensor: shape=(1037,), dtype=string, numpy=
array([b'darija', b'homa mkhbbyin chi haja, ana mti99en!',
       b'bayna homa tay7awlo ib9aw mbrrdin.', ..., b'', b'', b''],
      dtype=object)>

In [10]:
english[654]

'Bye'

In [11]:
darija[654]

'lay3awn'

In [12]:
dataset = tf.data.Dataset.from_tensor_slices((english, darija))

In [13]:
batch_size = 128
def batch_iterator():
    for i in range(0,len(df),batch_size):
        yield df[i:i+batch_size]["darija"]

In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [15]:
darija_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(),vocab_size=30000)

In [16]:
darija_tokenizer.save_pretrained("darija-tokenizer")

('darija-tokenizer\\tokenizer_config.json',
 'darija-tokenizer\\special_tokens_map.json',
 'darija-tokenizer\\vocab.txt',
 'darija-tokenizer\\added_tokens.json',
 'darija-tokenizer\\tokenizer.json')

In [17]:
seq_len = 64
tokens = darija_tokenizer(df['darija'].tolist(), max_length = seq_len,
                  truncation = True, padding='max_length',add_special_tokens=True,
                  return_tensors='np')

In [18]:
tokens['input_ids'][2]

array([   2, 3746,  653, 4286, 2027, 3704,    9,    3,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [19]:
df = pd.read_csv(r"C:\Users\adnane\Desktop\ORANGECHAT2\darija-data\imagenet_b_darija.csv", delimiter=',', encoding='utf8')
df.head()

,english,specific1_darija,specific2_darija,general_darija,darija_ar
0,kit_fox,NaN,ta3leb,t3leb,تعلب
1,English_setter,NaN,NaN,kelb,كلب
2,Siberian_husky,husky dial siberia,NaN,kelb,كلب
3,Australian_terrier,NaN,NaN,kelb,كلب
4,English_springer,NaN,NaN,kelb,كلب


In [20]:
import nltk 
from nltk import word_tokenize
from nltk.corpus import stopwords
import re
import string
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

In [22]:

def removePunctuation(x):
    x = x.lower()
    x = re.sub(r'[^\x00-\x7f]',r' ',x)
    x = x.replace('\r','')
    x = x.replace('\n','')
    x = x.replace('  ','')
    x = x.replace('\'','')
    return re.sub("["+string.punctuation+"]", " ", x)

In [25]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

#add words that aren't in the NLTK stopwords list
new_stopwords = ['and', 'at', 'so', 'er']
new_stopwords_list = stop_words.union(new_stopwords)

#remove words that are in NLTK stopwords list
not_stopwords = {'snake', 'lion', 'pigs'} 
final_stop_words = set([word for word in new_stopwords_list if word not in not_stopwords])

print(final_stop_words)

{'or', 'and', 'haven', 's', 'the', "you've", 'very', 'all', 'mightn', 'y', 'have', 'be', 'for', 'been', 'were', 'can', 'some', 'yours', 'we', 'other', 'so', 'these', "didn't", 'hers', "hasn't", 'yourself', 'you', 'doing', 'below', 'couldn', 'was', "shouldn't", 'has', 'against', 'itself', 'if', 'up', "weren't", 'with', "wasn't", 'whom', 'into', 'more', 'should', 'in', 'how', 'didn', 'd', 'ain', "she's", "mightn't", 'he', 'few', 'wouldn', 'don', 'before', 'to', 're', 'down', 'their', 'do', 'above', "shan't", 'while', 'by', 'where', "you'll", 'of', 'again', 'she', 'hasn', "should've", 'aren', 'herself', 'wasn', 'that', 'an', 'most', 'at', 'no', 'me', 'is', 'am', 'what', 'off', "mustn't", 'out', 'weren', 'they', 'himself', "wouldn't", 'ours', 'er', 'through', 'mustn', 'won', "hadn't", 'will', 'theirs', 'nor', 'then', 'which', 'being', 'm', 'its', 'now', "haven't", "don't", 'him', 'needn', 'isn', 'll', 'after', 'this', 'had', 'same', 'on', 'only', 'a', 'as', "couldn't", 'just', "you'd", 'no

In [26]:
def processText(x):
    x= removePunctuation(x)
    #x= removeStopwords(x)
    return x

In [29]:
from nltk.tokenize import sent_tokenize, word_tokenize
tin = pd.Series([word_tokenize(processText(x)) for x in df['english']])
tin.head(10)

0               [kit, fox]
1        [english, setter]
2        [siberian, husky]
3    [australian, terrier]
4      [english, springer]
5            [grey, whale]
6          [lesser, panda]
7          [egyptian, cat]
8                   [ibex]
9           [persian, cat]
dtype: object

In [34]:
from gensim.models import Word2Vec

sentences = [ "hurry and you will be late","a man bitten by a snake is afraid of rope", "he knocks and says who is there", "a friend in need is a friend indeed", "better luck next time"]

vocab = [s.encode('utf-8').split() for s in sentences]

voc_vec = word2vec.Word2Vec(vocab, min_count=1)

In [37]:
from gensim.models import word2vec
num_features = 300    # Word vector dimensionality                      
min_word_count = 4   # Minimum word count #Changed 40 to 4 only avoid build a vocabulary 1st                       
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words
model = word2vec.Word2Vec(tin, workers=num_workers, 
                          min_count = min_word_count,
                          window = context, sample = downsampling)

In [35]:
from gensim import utils
import logging
from timeit import default_timer
import threading
from six.moves import range
from six import itervalues, string_types
from gensim import matutils
from numpy import float32 as REAL, ones, random, dtype
from types import GeneratorType
from gensim.utils import deprecated
import os
import copy

In [36]:
def most_similar(self, positive=None, negative=None, topn=10, restrict_vocab=None, indexer=None):
        """Deprecated, use self.wv.most_similar() instead.
        Refer to the documentation for :meth:`~gensim.models.keyedvectors.WordEmbeddingsKeyedVectors.most_similar`.
        """
        return self.wv.most_similar(positive, negative, topn, restrict_vocab, indexer)

In [46]:
data = pd.read_csv(r'C:\Users\adnane\Desktop\ORANGECHAT2\darija-data\sentences2.csv')

In [47]:
data = data.dropna()

# Display the first few rows to understand the structure
data.head()

,english,darija
0,This is not a good start to the day,Hadi machi bedia zina l nhar
1,As you say.,klamk howa lkbir
2,"I ve got so much to do today, you know there a...","3ndi bezzaf maydar lyouma, rak 3arf lw9t mziye..."
3,Let's see if anything happens quickly.,nchoufo ila trat chi 7aja b zerba
4,I wish I had climbed the stairs,kantmenna koun tl3t f droj


In [49]:
from sklearn.model_selection import train_test_split

# Assuming the CSV has two columns: 'English' and 'Darija'
english_sentences = data['english'].tolist()
darija_sentences = data['darija'].tolist()

# Split the data into training, validation, and test sets
train_eng, temp_eng, train_dar, temp_dar = train_test_split(english_sentences, darija_sentences, test_size=0.2)
val_eng, test_eng, val_dar, test_dar = train_test_split(temp_eng, temp_dar, test_size=0.5)

In [50]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Tokenize the sentences
tokenizer_eng = Tokenizer()
tokenizer_eng.fit_on_texts(train_eng)
train_eng_seq = tokenizer_eng.texts_to_sequences(train_eng)
val_eng_seq = tokenizer_eng.texts_to_sequences(val_eng)

tokenizer_dar = Tokenizer()
tokenizer_dar.fit_on_texts(train_dar)
train_dar_seq = tokenizer_dar.texts_to_sequences(train_dar)
val_dar_seq = tokenizer_dar.texts_to_sequences(val_dar)

In [51]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Convert the data to lowercase and remove any special characters
data['english'] = data['english'].str.lower().str.replace('[^\w\s]', '', regex=True)
data['darija'] = data['darija'].str.lower().str.replace('[^\w\s]', '', regex=True)

# Add special tokens to the sentences
data['english'] = data['english'].apply(lambda x: 'starttoken ' + x + ' endtoken')
data['darija'] = data['darija'].apply(lambda x: 'starttoken ' + x + ' endtoken')

# Initialize tokenizers and add special tokens explicitly
english_tokenizer = Tokenizer(filters='')
darija_tokenizer = Tokenizer(filters='')

# Manually add the special tokens
english_tokenizer.fit_on_texts(['starttoken', 'endtoken'])
darija_tokenizer.fit_on_texts(['starttoken', 'endtoken'])

# Fit the tokenizers on the actual text data
english_tokenizer.fit_on_texts(data['english'])
darija_tokenizer.fit_on_texts(data['darija'])

# Convert text to sequences
english_sequences = english_tokenizer.texts_to_sequences(data['english'])
darija_sequences = darija_tokenizer.texts_to_sequences(data['darija'])



# Pad the sequences
max_eng = [len(seq) for seq in english_sequences]
max_english_length = 0
for i in max_eng:
    if i>max_english_length:
        max_english_length=i

max_dar = [len(seq) for seq in darija_sequences]
max_darija_length = 0
for i in max_dar:
    if i>max_darija_length:
        max_darija_length=i
        
english_sequences = pad_sequences(english_sequences, maxlen=max_english_length, padding='post')
darija_sequences = pad_sequences(darija_sequences, maxlen=max_darija_length, padding='post')

In [52]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_val, y_train, y_val = train_test_split(english_sequences, darija_sequences, test_size=0.2, random_state=42)

In [53]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# Define the encoder
encoder_inputs = Input(shape=(max_english_length,))
encoder_embedding = Embedding(len(english_tokenizer.word_index) + 1, 256, mask_zero=True)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(256, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Define the decoder
decoder_inputs = Input(shape=(max_darija_length,))
decoder_embedding = Embedding(len(darija_tokenizer.word_index) + 1, 256, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(len(darija_tokenizer.word_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Print the model summary
model.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 40)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 31)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 40, 256)              305152    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 31, 256)              443904    ['input_2[0][0]']             
                                                                                            

In [68]:
# Prepare target data by shifting the sequences
y_train_shifted = np.zeros_like(y_train)
y_train_shifted[:, :-1] = y_train[:, 1:]

y_val_shifted = np.zeros_like(y_val)
y_val_shifted[:, :-1] = y_val[:, 1:]

# Train the model
model.fit([X_train, y_train], y_train_shifted, validation_data=([X_val, y_val], y_val_shifted), epochs=50, batch_size=4)

Epoch 1/50
146/146 [==============================] - 8s 54ms/step - loss: 0.0129 - val_loss: 8.1453
Epoch 2/50
146/146 [==============================] - 8s 54ms/step - loss: 0.0112 - val_loss: 8.1810
Epoch 3/50
146/146 [==============================] - 8s 52ms/step - loss: 0.0095 - val_loss: 8.2122
Epoch 4/50
146/146 [==============================] - 8s 51ms/step - loss: 0.0093 - val_loss: 8.2367
Epoch 5/50
146/146 [==============================] - 8s 52ms/step - loss: 0.0085 - val_loss: 8.2624
Epoch 6/50
146/146 [==============================] - 8s 52ms/step - loss: 0.0076 - val_loss: 8.2830
Epoch 7/50
146/146 [==============================] - 8s 53ms/step - loss: 0.0072 - val_loss: 8.3069
Epoch 8/50
146/146 [==============================] - 8s 52ms/step - loss: 0.0071 - val_loss: 8.3276
Epoch 9/50
146/146 [==============================] - 8s 52ms/step - loss: 0.0065 - val_loss: 8.3525
Epoch 10/50
146/146 [==============================] - 8s 52ms/step - loss: 0.0066 - val_lo

In [69]:
# Evaluate the model on the validation set
loss = model.evaluate([X_val, y_val], y_val_shifted)
print(f'Validation Loss: {loss}')

5/5 [==============================] - 0s 36ms/step - loss: 9.2050
Validation Loss: 9.20496654510498


In [71]:
def translate(sentence):
    # Preprocess the sentence
    sentence = 'starttoken ' + sentence.lower().replace('[^\w\s]', '') + ' endtoken'
    sequence = english_tokenizer.texts_to_sequences([sentence])
    sequence = pad_sequences(sequence, maxlen=max_english_length, padding='post')

    # Encode the sentence
    states = encoder_model.predict(sequence)
    
    # Initialize the target sequence with the start token
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = darija_tokenizer.word_index['starttoken']
    
    translated_sentence = []
    
    for _ in range(max_darija_length):
        # Predict the next word
        output_tokens, h, c = decoder_model.predict([target_seq] + states)
        
        # Get the highest probability word index
        word_id = np.argmax(output_tokens[0, -1, :])
        word = darija_tokenizer.index_word.get(word_id, '')
        
        if word == 'endtoken':
            break
        
        translated_sentence.append(word)
        
        # Update the target sequence and states
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = word_id
        states = [h, c]

    return ' '.join(translated_sentence)

# Define the encoder model for prediction
encoder_model = Model(encoder_inputs, encoder_states)

# Define the decoder model for prediction
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Translate a sample sentence
print(translate('how i can help you?').replace('starttoken ',''))

1/1 [==============================] - 0s 32ms/step
katban lia


In [ ]:
# Save the entire model
model.save('translation_model.h5')

In [ ]:
import pickle

# Save English tokenizer
with open('english_tokenizer.pkl', 'wb') as f:
    pickle.dump(english_tokenizer, f)

# Save Darija tokenizer
with open('darija_tokenizer.pkl', 'wb') as f:
    pickle.dump(darija_tokenizer, f)

# Save maximum sequence lengths
with open('max_seq_lengths.pkl', 'wb') as f:
    pickle.dump({'max_english_length': max_english_length, 'max_darija_length': max_darija_length}, f)